<a href="https://colab.research.google.com/github/qidopox/Deep_Learning_for_image_processing_practice/blob/main/DL_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to Deep Learning for image processing practice

Convolutional Neural network (CNN) was usually used for image processing. Common applications include image classification, segmentation and denoising. In this practice, we are going to train a neural network for classifying handwritten numbers （MNIST dataset）. We are going to use two different trained neural networks for two separate tasks - image segmentation and denoising.

##Exercise 1: Train a neural network for classifying handwritten numbers

In this practice, we will use MNIST database. The MNIST database (Modified National Institute of Standards and Technology database) is a collection of handwritten digits. For details of the dataset, please see https://paperswithcode.com/dataset/mnist


We first import the relevant python packages. In this case, we will use tensorflow and keras to build and train neural networks. The matlotlib package is for figure plotting. The datatime package is to provide access to date and time.

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
import matplotlib.pyplot as plt
import datetime


MNIST dataset was included in the keras dataset and thus can be loaded directly from keras.

As tensorflow takes float32 as the input, we need to convert the input images, which are in uint8 format, to float32.

In [ ]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = tf.cast(x_train, tf.float32) / 255
x_test = tf.cast(x_test, tf.float32) / 255

Printing out the sizes of the training dataset and test dataset. There are 60,000 examples in the training dataset and 10,000 in the test dataset. The image inputs are in the sizes of 28 by 28.

In [ ]:
print('training data input shape:',x_train.shape,'\n training data label shape',y_train.shape,'\n test data input shape:',x_test.shape,'\n test data label shape:',y_test.shape)

Plotting out an example from the training dataset and check its corresponding label is correct.

In [ ]:
i = 0
plt.imshow(x_train[i,:,:])
plt.show()
print('label of the figure is ', y_train[i])

Construct a neural network with an input the same size as the images and an output equals to 10 which corresponds to the 10 different digits, 0-9.

In [ ]:
inputs = inputs = keras.Input(shape=(x_train.shape[1], x_train.shape[2]))
x = keras.layers.Flatten()(inputs)
x = keras.layers.Dense(128, activation=tf.nn.relu)(x)
outputs = keras.layers.Dense(10, activation=tf.nn.softmax)(x)
model= keras.Model(inputs=inputs, outputs=outputs,name='mnist_classification_fully_corrected')

We can print out a summary of the neural network.

In [ ]:
model.summary()

We select "Adam" as the optimisation algorithm for the neural network training. The loss function is "SparseCategoricalCrossentropy". We also use tensorboard to monitor the training process.

For details of "Adam", please find https://arxiv.org/abs/1412.6980

For details of "SparseCategoricalCrossentropy", please find https://www.tensorflow.org/api_docs/python/tf/keras/losses/SparseCategoricalCrossentropy

In [ ]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-3),
              loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],)
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

We fit x_train dataset as the input training dataset and y_train as the output training dataset. We select to train a maximum of 30 epochs. We fit x_test dataset as the input validation dataset and y_test as the output validation dataset.

By running this cell, you will start to train your neural network.

In [ ]:
model.fit(
    x=x_train,y=y_train,
    epochs=30,
    validation_data=(x_test,y_test),
    callbacks=[tensorboard_callback],
)

Open tensorboard to monitor the network training.

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs/fit

The trained network model is saved in the folder "models" as a h5 file.

In [ ]:
model.save('./models/'+model.name+'_.h5')

Loading the trained model. Using the trained model to classify the digit from handwritten number.

In [ ]:
model = keras.models.load_model('./models/'+model.name+'_.h5', compile=False)
i = 1
y_pred=model.predict(x_test[i,:,:].reshape([1,28,28,1]))

print('label:',y_test[i],'\n prediction:',y_pred)

##Exercise 2: Train a convolutional neural network working on MNIST dataset for classifying handwritten numbers

Let's now construct a CNN for the same task. Please compare the architectures and the performances of the two types of networks.

In [ ]:
inputs = keras.Input(shape=(x_train.shape[1], x_train.shape[2],1))
x = keras.layers.Conv2D(4,(3, 3),activation=tf.nn.relu,
                  kernel_initializer="glorot_uniform",
                  padding="same",name="conv1",)(inputs)
x = keras.layers.MaxPooling2D((2, 2), strides=(2, 2), name="pool1")(x)
x = keras.layers.Conv2D(8,(3, 3),activation=tf.nn.relu,
                  kernel_initializer="glorot_uniform",
                  padding="same",name="conv2",)(x)
x = keras.layers.MaxPooling2D((2, 2), strides=(2, 2), name="pool2")(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(16, activation=tf.nn.relu)(x)
outputs = keras.layers.Dense(10, activation=tf.nn.softmax)(x)
model_CNN = keras.Model(inputs=inputs, outputs=outputs)


In [ ]:
model_CNN.summary()

In [ ]:
model_CNN.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-3),
              loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],)
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
model_CNN.fit(
    x=x_train,y=y_train,
    epochs=30,
    validation_data=(x_test,y_test),
    callbacks=[tensorboard_callback],
)

 Please take a look at the loss function plot (epoch_loss) on the tensorboard. What are the differences of the two plots? Why do you think the differences?
 If you repeat the process and retrain the network, do you obtain the same result plots on tensorboard?

##Exercise 3: Train a U-net for image segmentation through transfer learning

In this practice, we use the Oxford-IIIT Pet Dataset (Parkhi et al, 2012). The dataset consists of images of 37 pet breeds. Please see details here https://www.robots.ox.ac.uk/%7Evgg/data/pets/

This Exercise was adapted from the tensorflow tutorial https://www.tensorflow.org/tutorials/images/segmentation



In [1]:
import tensorflow_datasets as tfds

If you work on this practice in the same runtime as the first practice, you will not need to re-import the following packages.

In [2]:
import tensorflow as tf
import tensorflow.keras as keras
import matplotlib.pyplot as plt
import datetime

Load the Oxford-IIIT Pet Dataset. It may take a while.

In [ ]:
dataset, info = tfds.load('oxford_iiit_pet:3.*.*', with_info=True)

Print the data info.

In [ ]:
print(info)

Prepare the data for batch training. There are two functions and one class we defined here:

1. Normalise the data so that the pixel values of the images fall in the range of [0,1]. The pixel values in the masks for image segmentations were labeled either {1,2,3}. It is easier to work in python if the labels are {0,1,2} and thus we deduct 1 to all pixels in the masks.

2. Load the images. Tensorflow dataset has a non-specified image size that allow users to define. We set the input images and masks to have sizes of 128 by 128.

3. Data augmentation. We do some simple data augmentation by flip and rotate the images and masks.

Finally we define training batch and test batch to feed into network training.

* Note that for those who are not too familiar with python, *def* and *class* can be seen as two styles of python programming and differences were subtle. *def* is task oriented and *class* is data oriented.

In [35]:
def normalize(input_image, input_mask):
  input_image = tf.cast(input_image, tf.float32) / 255.0
  input_mask -= 1
  return input_image, input_mask

def load_image(datapoint):
  input_image = tf.image.resize(datapoint['image'], (128, 128))
  input_mask = tf.image.resize(
    datapoint['segmentation_mask'],
    (128, 128),
    method = tf.image.ResizeMethod.NEAREST_NEIGHBOR,
  )

  input_image, input_mask = normalize(input_image, input_mask)

  return input_image, input_mask

TRAIN_LENGTH = info.splits['train'].num_examples
BATCH_SIZE = 64
BUFFER_SIZE = 1000
STEPS_PER_EPOCH = TRAIN_LENGTH // BATCH_SIZE

train_images = dataset['train'].map(load_image, num_parallel_calls=tf.data.AUTOTUNE)
test_images = dataset['test'].map(load_image, num_parallel_calls=tf.data.AUTOTUNE)

class Augment(tf.keras.layers.Layer):
  def __init__(self, seed=(42,15,28)):
    super().__init__()
    # both use the same seed, so they'll make the same random changes.
    self.augment_inputs = tf.keras.layers.RandomFlip(mode="horizontal", seed=seed[0])
    self.augment_labels = tf.keras.layers.RandomFlip(mode="horizontal", seed=seed[0])

    self.augment_inputs = tf.keras.layers.RandomFlip(mode="vertical", seed=seed[1])
    self.augment_labels = tf.keras.layers.RandomFlip(mode="vertical", seed=seed[1])

    self.augment_inputs = tf.keras.layers.RandomRotation(factor=0.5, seed=seed[2])
    self.augment_labels = tf.keras.layers.RandomRotation(factor=0.5, seed=seed[2])


  def call(self, inputs, labels):
    inputs = self.augment_inputs(inputs)
    labels = self.augment_labels(labels)
    return inputs, labels

train_batches = (
    train_images
    .cache()
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE)
    .repeat()
    .map(Augment())
    .prefetch(buffer_size=tf.data.AUTOTUNE))

test_batches = test_images.batch(BATCH_SIZE)

Now let's take a look of the training input images and their corresponding masks.

In [ ]:
def display(display_list):
  plt.figure(figsize=(15, 15))

  title = ['Input Image', 'True Mask', 'Predicted Mask']

  for i in range(len(display_list)):
    plt.subplot(1, len(display_list), i+1)
    plt.title(title[i])
    plt.imshow(tf.keras.utils.array_to_img(display_list[i]))
    plt.axis('off')
  plt.show()

for images, masks in train_batches.take(10):
  sample_image, sample_mask = images[0], masks[0]
  display([sample_image, sample_mask])